In [31]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px


In [75]:
df = pd.read_csv('data/78858099999.csv', parse_dates=["DATE"],index_col=["DATE"] )
df[['Dir','DirQ', 'Obs', 'Speed','SpeedQ']] = df['WND'].str.split(',', expand=True)
df['Speed'] = df['Speed'].astype(int)
df['Dir'] = df['Dir'].astype(int)
df = df.drop(df[(df.Speed == 9999)].index)
df = df.drop(df[(df.Dir == 999)].index)

df2 = pd.read_csv('data/78859099999.csv', parse_dates=["DATE"],index_col=["DATE"] )
df2[['Dir','DirQ', 'Obs', 'Speed','SpeedQ']] = df2['WND'].str.split(',', expand=True)
df2['Speed'] = df2['Speed'].astype(int)
df2['Dir'] = df2['Dir'].astype(int)
df2 = df2.drop(df2[(df2.Speed == 9999)].index)
df2 = df2.drop(df2[(df2.Dir == 999)].index)


df3 = pd.read_csv('data/78873000400.csv', parse_dates=["DATE"],index_col=["DATE"] )
df3[['Dir','DirQ', 'Obs', 'Speed','SpeedQ']] = df3['WND'].str.split(',', expand=True)
df3['Speed'] = df3['Speed'].astype(int)
df3['Dir'] = df3['Dir'].astype(int)
df3 = df3.drop(df3[(df3.Speed == 9999)].index)
df3 = df3.drop(df3[(df3.Dir == 999)].index)


df4 = pd.read_csv('data/78873000400.csv', parse_dates=["DATE"],index_col=["DATE"] )
df4[['Dir','DirQ', 'Obs', 'Speed','SpeedQ']] = df4['WND'].str.split(',', expand=True)
df4['Speed'] = df4['Speed'].astype(int)
df4['Dir'] = df4['Dir'].astype(int)
df4 = df4.drop(df4[(df4.Speed == 9999)].index)
df4 = df4.drop(df4[(df4.Dir == 999)].index)


In [65]:
import numpy as np
def wind_dir_speed_freq(boundary_lower_speed, boundary_higher_speed, boundary_lower_direction, boundary_higher_direction):
    # mask for wind speed column
    log_mask_speed = (wind_rose_data[:, 0] >= boundary_lower_speed) & (wind_rose_data[:, 0] < boundary_higher_speed)
    # mask for wind direction
    log_mask_direction = (wind_rose_data[:, 1] >= boundary_lower_direction) & (
                wind_rose_data[:, 1] < boundary_higher_direction)

    # application of the filter on the wind_rose_data array
    return wind_rose_data[log_mask_speed & log_mask_direction]    # application of the filter on the wind_rose_data array

# Creating a pandas dataframe with 8 wind speed bins for each of the 16 wind directions.
# dataframe structure: direction | strength | frequency (radius)

wind_rose_df = pd.DataFrame(np.zeros((16 * 9, 3)), index=None, columns=('direction', 'strength', 'frequency'))

directions = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
directions_deg = np.array([0, 22.5, 45, 72.5, 90, 112.5, 135, 157.5, 180, 202.5, 225, 247.5, 270, 292.5, 315, 337.5])
speed_bins = ['0-9 m/s', '10-19 m/s', '20-29 m/s', '30-39 m/s', '40-49 m/s', '50-59 m/s', '60-69 m/s', '70-79 m/s', '>80 m/s']

# filling in the dataframe with directions and speed bins
wind_rose_df.direction = directions * 9
wind_rose_df.strength = np.repeat(speed_bins, 16)

#creating a multiindex dataframe with frequencies

idx = pd.MultiIndex.from_product([speed_bins,
                                  directions_deg],
                                 names=['wind_speed_bins', 'wind_direction_bins'])
col = ['frequency']
frequencies_df = pd.DataFrame(0, idx, col)

def build_wind_rose(data_frame, title):
    wind_rose_data = data_frame[['Speed', 'Dir']].to_numpy()

    # distance between the centre of the bin and its edge
    step = 11.25

    # converting data between 348.75 and 360 to negative
    for i in range(len(wind_rose_data)):
        if directions_deg[-1] + step <= wind_rose_data[i, 1] and wind_rose_data[i, 1] < 360:
            wind_rose_data[i, 1] = wind_rose_data[i, 1] - 360

    # determining the direction bins
    bin_edges_dir = directions_deg - step
    bin_edges_dir = np.append(bin_edges_dir, [directions_deg[-1] + step])

    # determining speed bins ( the last bin is 50 as above those speeds the outliers were removed for the measurements)
    threshold_outlier_rm = 100
    bin_edges_speed = np.array([0, 10, 20, 30, 40, 50, 60, 70, 80, threshold_outlier_rm])

    frequencies = np.array([])
    # loop selecting given bins and calculating frequencies
    for i in range(len(bin_edges_speed) - 1):
        for j in range(len(bin_edges_dir) - 1):
            bin_contents = wind_dir_speed_freq(bin_edges_speed[i], bin_edges_speed[i + 1], bin_edges_dir[j],
                                               bin_edges_dir[j + 1])

            # applying the filtering function for every bin and checking the number of measurements
            bin_size = len(bin_contents)
            frequency = bin_size / len(wind_rose_data)

            #obtaining the final frequencies of bin
            frequencies = np.append(frequencies, frequency)

    # updating the frequencies dataframe
    frequencies_df.frequency = frequencies * 100  # [%]
    wind_rose_df.frequency = frequencies * 100  # [%]

    # calling the PLOT function
    """
    PLOTTING THE ROSES
    """
    # fig_wind_rose = wind_rose_fig(frequencies_df,
    #                               title='<b>WRF Data: Wind Speed Distribution</b>',
    #                               filename='fig_wind_rose_WRF.png',
    #                               open_bool=False)

    import plotly.graph_objects as go
    from plotly.offline import plot
    import kaleido

    fig = go.Figure()

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('0-9 m/s'), 'frequency'],
        name='0-9 m/s',
        marker_color='#482878'))

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('10-19 m/s'), 'frequency'],
        name='10-19 m/s',
        marker_color='#3e4989'))

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('20-29 m/s'), 'frequency'],
        name='20-29 m/s',
        marker_color='#31688e'))

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('30-39 m/s'), 'frequency'],
        name='30-39 m/s',
        marker_color='#26828e'))

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('40-49 m/s'), 'frequency'],
        name='40-49 m/s',
        marker_color='#1f9e89'))

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('50-59 m/s'), 'frequency'],
        name='50-59 m/s',
        marker_color='#35b779'))

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('60-69 m/s'), 'frequency'],
        name='60-69 m/s',
        marker_color='#6ece58'))

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('70-79 m/s'), 'frequency'],
        name='70-79 m/s',
        marker_color='#b5de2b'))

    fig.add_trace(go.Barpolar(
        r=frequencies_df.loc[('>80 m/s'), 'frequency'],
        name='>80 m/s',
        marker_color='#fde725'))

    fig.update_traces(
        text=['North', 'NNE', 'NE', 'ENE', 'East', 'ESE', 'SE', 'SSE', 'South', 'SSW', 'SW', 'WSW', 'West', 'WNW', 'NW', 'NNW'])

    fig.update_layout(
        title=title,
        title_font_size=26,
        title_x=0.463,
        legend_font_size=18,
        polar_radialaxis_ticksuffix='%',
        polar_angularaxis_rotation=90,
        polar_angularaxis_direction='clockwise',
        polar_angularaxis_tickmode='array',
        polar_angularaxis_tickvals=[0, 22.5, 45, 72.5, 90, 112.5, 135, 157.5, 180, 202.5, 225, 247.5, 270, 292.5, 315, 337.5],
        polar_angularaxis_ticktext=['<b>North</b>', 'NNE', '<b>NE</b>', 'ENE', '<b>East</b>', 'ESE', '<b>SE</b>', 'SSE', '<b>South</b>', 'SSW', '<b>SW</b>', 'WSW', '<b>West</b>', 'WNW', '<b>NW</b>', 'NNW'],
        polar_angularaxis_tickfont_size=22,
        polar_radialaxis_tickmode='linear',
        polar_radialaxis_angle=45,
        polar_radialaxis_tick0=5,
        polar_radialaxis_dtick=5,
        polar_radialaxis_tickangle=100,
        polar_radialaxis_tickfont_size=14)

    #fig.write_image(get_fig_dir() + filename, width=1000, height=800)
    return fig.show()

In [66]:
build_wind_rose(df,'ROBERT L BRADSHAW, SC')

In [69]:
build_wind_rose(df2,'VANCE WINKWORTH AMORY INTL, SC')

In [76]:
build_wind_rose(df4, 'F D ROOSEVELT AIRPORT, NL')


In [40]:
fig = px.line(df.reset_index(), x='DATE', y="Speed", title='Wind Speed')
fig.show()

In [72]:
df3

,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,...,OC1,OD1,OD2,REM,EQD,Dir,DirQ,Obs,Speed,SpeedQ
DATE,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,1010099999,4,69.2925,16.144167,13.1,"ANDOYA, NO",FM-12,99999,V020,"140,1,N,0072,1",...,NaN,NaN,NaN,SYN004BUFR,NaN,140,1,N,72,1
2021-01-01 00:20:00,1010099999,4,69.2925,16.144167,13.1,"ANDOYA, NO",FM-15,99999,V020,"150,1,N,0082,1",...,NaN,NaN,NaN,MET060METAR ENAN 010020Z AUTO 15016KT 9999 FEW...,NaN,150,1,N,82,1
2021-01-01 00:50:00,1010099999,4,69.2925,16.144167,13.1,"ANDOYA, NO",FM-15,99999,V020,"150,1,N,0082,1",...,NaN,NaN,NaN,MET054METAR ENAN 010050Z AUTO 15016KT 9999 NCD...,NaN,150,1,N,82,1
2021-01-01 01:20:00,1010099999,4,69.2925,16.144167,13.1,"ANDOYA, NO",FM-15,99999,V020,"150,1,N,0077,1",...,NaN,NaN,NaN,MET054METAR ENAN 010120Z AUTO 15015KT 9999 NCD...,NaN,150,1,N,77,1
2021-01-01 01:50:00,1010099999,4,69.2925,16.144167,13.1,"ANDOYA, NO",FM-15,99999,V020,"150,1,N,0088,1",...,NaN,NaN,NaN,MET054METAR ENAN 010150Z AUTO 15017KT 9999 NCD...,NaN,150,1,N,88,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 22:20:00,1010099999,4,69.2925,16.144167,13.1,"ANDOYA, NO",FM-15,99999,V020,"010,1,N,0072,1",...,NaN,NaN,NaN,MET077METAR ENAN 312220Z AUTO 01014KT 9999 -FZ...,NaN,10,1,N,72,1
2021-12-31 22:50:00,1010099999,4,69.2925,16.144167,13.1,"ANDOYA, NO",FM-15,99999,V020,"040,1,N,0093,1",...,NaN,NaN,NaN,MET081METAR ENAN 312250Z AUTO 04018KT 9999 FZU...,D01 0ADE735,40,1,N,93,1
2021-12-31 23:00:00,1010099999,4,69.2925,16.144167,13.1,"ANDOYA, NO",FM-12,99999,V020,"030,1,N,0087,1",...,"0128,1","4,01,0128,1,137","9,99,0113,1,999",SYN004BUFR,NaN,30,1,N,87,1


In [15]:
def histogram_wind_dir_fig(data, name, color, title, xaxis_title, yaxis_title, filename, open_bool):
    fig = go.Figure()
    fig.add_trace(go.Histogram(
        x=data,
        histnorm='percent',
        name=name,  # name used in legend and hover labels
        xbins=dict(  # bins used for histogram
            size=3),
        marker_color=color))

    fig.update_layout(
        title_text=title,  # title of plot
        title_x=0.5,
        title_font_size=24,
        xaxis=dict(title=xaxis_title,
                   titlefont_size=20,
                   tickfont_size=16,
                   tickvals=[0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360],
                   ticktext=['N', 30, 60, 'E', 120, 150, 'S', 210, 240, 'W', 300, 330, 'N']),  # xaxis label
        yaxis=dict(title=yaxis_title,
                   titlefont_size=20,
                   tickfont_size=16),  # yaxis label
        bargap=0.2, )  # gap between bars of adjacent location coordinates

    fig.show(renderer='png', width=1200, height=500)
    return plot(fig, auto_open=open_bool)